# Data Acquisition
### Table of Contents
- [Requirements](#requirements)
- [Obtaining Historical Yield Data](#obtaining-historical-yield-data)
- [Obtaining Historical Price Received Data](#obtaining-historical-price-received-data)
- [Obtaining Historical Weather Data](#obtaining-historical-weather-data)

## Requirements

In [1]:
import requests
import urllib.parse
import pandas as pd
from io import StringIO

## Obtaining Historical Yield Data
#### (Grain & Silage)

In [2]:
# retrieving the access key for the USDA NASS API

print('\nLoading...\n')

with open('../../keys/api_key.txt', 'r') as file: # file path appears as `keys/api_key.txt` in `acquisition.py`
    usda_nass_key = file.read()


Loading...



In [3]:
# assignment of variables for later use

states = ['ILLINOIS', 'INDIANA', 'IOWA', 'MINNESOTA', 'MISSOURI', 'NEBRASKA']

parameters = {
    'key': usda_nass_key,
    'source_desc': 'SURVEY',
    'sector_desc': 'CROPS',
    'group_desc': 'FIELD CROPS',
    'commodity_desc': 'CORN',
    'statisticcat_desc': 'YIELD',
    'agg_level_desc': 'STATE',
    'format': 'csv'
}

In [4]:
## encoding query parameters according to URL standards
encoded_parameters = urllib.parse.urlencode(parameters)

## establishing the URL base for the query
base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

## constructing the API query from the URL base and the URL encoded parameters
query = base_url+'?'+encoded_parameters

## fetching the query response
response = requests.get(query)

In [5]:
## if the query was successful:
if response.status_code == 200:

    ### convert query response to text
    data = StringIO(response.text)

    ### convert response text to a pandas dataframe
    yield_raw = pd.read_csv(data)

    ### narrow the obtained dataset to only the necessary columns to reduce storage waste
    yield_raw = yield_raw[['year', 'util_practice_desc', 'state_name', 'reference_period_desc', 'Value']]

    ### we are only interested in the yearly recorded yield, not forecasts or monthly estimates
    yield_raw = yield_raw[yield_raw['reference_period_desc'] == 'YEAR']

    ### save the dataframe as a local CSV
    yield_raw.to_csv('../../data/raw/yield_raw.csv', index=False) # file path appears as `data/raw/yield_raw.csv` in `acquisition.py`
    print('Data saved as yield_raw.csv')

## error handling
else:
    print(f'Request failed with status code {response.status_code}')

Data saved as yield_raw.csv


## Obtaining Historical Price Received Data
#### (By Month as well as Marketing Year)

In [6]:
# since the historical price received data is too large to be fetched in one query, we fetch each state's data separately

print('\nLoading...\n')

for i, state in enumerate(states):

    ## assignment of variables for later use
    parameters = {
        'key': usda_nass_key,
        'source_desc': 'SURVEY',
        'sector_desc': 'CROPS',
        'group_desc': 'FIELD CROPS',
        'commodity_desc': 'CORN',
        'statisticcat_desc': 'PRICE RECEIVED',
        'agg_level_desc': 'STATE',
        'state_name':state,
        'format': 'csv'
    }

    ## encoding query parameters according to URL standards
    encoded_parameters = urllib.parse.urlencode(parameters)

    ## establishing the URL base for the query
    base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

    ## constructing the API query from the URL base and the URL encoded parameters
    query = base_url+'?'+encoded_parameters

    ## fetching the query response
    response = requests.get(query)

    ## if the query was successful:
    if response.status_code == 200:

        ### for the first state, ILLINOIS, create the `price_received_raw` dataframe
        if i == 0:
            data = StringIO(response.text)
            price_received_raw = pd.read_csv(data)
            price_received_raw = price_received_raw[['year', 'state_name', 'reference_period_desc', 'Value']]

        ### for every state thereafter, create a temporary dataframe with the 'next' state's data and merge it with the `price_received_raw` dataframe
        else:
            data = StringIO(response.text)
            temp_df = pd.read_csv(data)
            temp_df = temp_df[['year', 'state_name', 'reference_period_desc', 'Value']]
            price_received_raw = pd.concat([price_received_raw, temp_df])

    ## error handling
    else:
        print(f'Request failed for {state} with status code {response.status_code}')


Loading...



In [7]:
# save the dataframe as a local CSV

price_received_raw.to_csv('../../data/raw/price_received_raw.csv', index=False) # file path appears as `data/raw/price_received_raw.csv` in `acquisition.py`
print('Data saved as price_received_raw.csv')

Data saved as price_received_raw.csv


## Obtaining Historical Weather Data
#### (Average/Maximum/Minimum Temperature, Total Precipitation, & PDSI by Month from April through November)

In [8]:
# assignment of variables for later use

print('\nLoading...\n')

months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
state_numbers = [11, 12, 13, 21, 23, 25]


Loading...



In [9]:
## create an empty dataframe template of years and states
years = list(range(1895, 2025))*6
years.sort()
list_temp = list(range(0, 6))*130
df_empty = pd.DataFrame({'year':years, 'state':list_temp})
df_empty['state'] = df_empty['state'].replace({i: states[i] for i in range(len(states))})

## create a copy of the empty dataframe template
weather_raw = df_empty.copy()

## iterate through each state, variable, & month combination
for s, state_number in enumerate(state_numbers):
    for variable in ['tavg', 'tmax', 'tmin', 'pcp', 'pdsi']:
        for m, month in enumerate(months):

            ### find the state
            state = states[s]

            ### skip the appropriate number of rows when reading the CSV
            skiprows = 3 if variable == 'pdsi' else 4

            ### construct the appropriate CSV link
            link = f'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/{state_number}/{variable}/12/{m+1}/1895-2024.csv?base_prd=true&begbaseyear=1895&endbaseyear=2000'

            ### create the temporary dataframe
            temp = pd.read_csv(link, skiprows=skiprows)

            ### reformat the `Date` column to be YYYY
            temp['Date'] = temp['Date'].apply(lambda x: int(str(x)[:4]))

            ### rename `Date` to `year`
            temp.rename(columns={'Date':'year'}, inplace=True)

            ### rename `Value` columns to `{month}-{var}`
            temp.rename(columns={'Value':f'{month}_{variable}'}, inplace=True)

            ### map the `temp` values to `weather_raw`
            weather_raw.loc[weather_raw['state'] == state, f'{month}_{variable}'] = weather_raw.loc[weather_raw['state'] == state, 'year'].map(temp.set_index('year')[f'{month}_{variable}'])

In [10]:
# save the dataframe as a local CSV

weather_raw.to_csv('../../data/raw/weather_raw.csv', index=False) # file path appears as `data/raw/weather_raw.csv` in `acquisition.py`
print('Data saved as weather_raw.csv')

Data saved as weather_raw.csv
